##### Copyright 2024 Google LLC.

In [9]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

### Install dependencies

In [10]:
!pip install -qU 'google-generativeai>=0.7.0'

In [11]:
import google.generativeai as genai
from google.generativeai import caching

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [12]:
from google.colab import userdata

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

This example will be based on the transcript from the Apollo 11 mission.

Start by downloading that transcript.

In [13]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


Now upload the transcript using the [File API](../quickstarts/File_API.ipynb).

In [14]:
document = genai.upload_file(path="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the [`system_instruction`](../quickstarts/System_instructions.ipynb) has been set, and the document was provided in the prompt.

In [15]:
# Note that caching requires a frozen model, e.g. one with a `-001` suffix.
model_name = "gemini-1.5-flash-002"
import google.generativeai as genai
from google.generativeai import caching
apollo_cache = caching.CachedContent.create(
    model=model_name,
    system_instruction="You are an expert at analyzing transcripts.",
    contents=[document],
)

apollo_cache

CachedContent(
    name='cachedContents/54q5b096l7d8',
    model='models/gemini-1.5-flash-002',
    display_name='',
    usage_metadata={
        'total_token_count': 323383,
    },
    create_time=2025-02-21 11:47:32.618223+00:00,
    update_time=2025-02-21 11:47:32.618223+00:00,
    expire_time=2025-02-21 12:47:31.275879+00:00
)

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [16]:
import datetime

apollo_cache.update(ttl=datetime.timedelta(hours=2))
apollo_cache

CachedContent(
    name='cachedContents/54q5b096l7d8',
    model='models/gemini-1.5-flash-002',
    display_name='',
    usage_metadata={
        'total_token_count': 323383,
    },
    create_time=2025-02-21 11:47:32.618223+00:00,
    update_time=2025-02-21 11:47:33.191626+00:00,
    expire_time=2025-02-21 13:47:33.131614+00:00
)

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [17]:
apollo_model = genai.GenerativeModel.from_cached_content(cached_content=apollo_cache)

response = apollo_model.generate_content("Find a lighthearted moment from this transcript")
print(response.text)

One lighthearted moment occurs on page 20, lines 50-51:

> 00 03 53 05 CDR
> And, Houston, you, might be interested that out my left-hand window right now, I can observe the entire continent of North America, Alaska, and over the Pole, down to the Yucatan Peninsula, Cuba, northern part of South America, and then I run out of window.
>
> 00 03 53 27 CC
> Roger. We copy.

While not explicitly humorous, Armstrong's deadpan delivery of the observation of seeing the entire continent of North America, then adding "and then I run out of window," creates a subtly comedic effect.  The understated nature of the statement, juxtaposed with the grandeur of the view, lends itself to lighthearted interpretation.



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [18]:
response.usage_metadata

prompt_token_count: 323391
cached_content_token_count: 323383
candidates_token_count: 185
total_token_count: 323576

You can ask new questions of the model, and the cache is reused.

In [19]:
chat = apollo_model.start_chat()
response = chat.send_message("Give me a quote from the most important part of the transcript.")
print(response.text)

The most important part of the transcript is the moment of the Apollo 11 lunar module's landing.  The quote from that moment is:

**"Houston, Tranquility Base here. The Eagle has landed."**



In [20]:
response = chat.send_message("What was recounted after that?")
print(response.text)

Immediately following the iconic "The Eagle has landed" transmission, the following events and communications were recounted in the transcript:

* **Confirmation and relief from Mission Control:** Houston responded with confirmation ("Roger, Tranquility. We copy you on the ground. You got a bunch of guys about to turn blue. We're breathing again. Thanks a lot.") expressing immense relief after the tense moments leading up to the landing.

* **Procedural updates and confirmations from the lunar module (LM):**  Armstrong and Aldrin proceeded with post-landing procedures, reporting actions like deploying the master arm switch, venting oxidizer, resetting engine stop, and checking various systems.

* **Communication with the command module (CM):**  There was brief communication with Michael Collins in the command module, who expressed his excitement ("Fantastic").

* **Reports of the landing site:**  Armstrong and Aldrin provided initial descriptions of the lunar surface, noting its fine, 

In [21]:
response.usage_metadata

prompt_token_count: 323450
cached_content_token_count: 323383
candidates_token_count: 353
total_token_count: 323803

As you can see, among the 323455 tokens, 323383 were cached (and thus less expensive) and only 236 were from the prompt.

Since the cached tokens are cheaper than the normal ones, it means this prompt was 75% cheaper that if you had not used caching. Check the [pricing here](https://ai.google.dev/pricing) for the up-to-date discount on cached tokens.

## Counting tokens

The `GenerativeModel` object can be used to count the tokens of a request in the same manner as a direct, uncached, model.

In [22]:
apollo_model.count_tokens("How many people are involved in this transcript?")

total_tokens: 323392
cached_content_token_count: 323383

## Delete the cache

The cache has a small recurring storage cost (cf. [pricing](https://ai.google.dev/pricing)) so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 2h.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [23]:
print(apollo_cache.name)
apollo_cache.delete()

cachedContents/54q5b096l7d8


## Next Steps
### Useful API references:

If you want to know more about the caching API, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

### Continue your discovery of the Gemini API

Check the File API notebook to know more about that API. The [vision capabilities](../quickstarts/Video.ipynb) of the Gemini API are a good reason to use the File API and the caching.
The Gemini API also has configurable [safety settings](../quickstarts/Safety.ipynb) that you might have to customize when dealing with big files.
